In [31]:
import numpy as np
import pandas as pd
import matplotlib
import IPython
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
DATA_PATH = "MADReSS_Challenge_2022/data/train/"
SAMPLES = "MADReSS_Challenge_2022/data/sample/"

In [32]:
# load all features
training_df = pd.read_csv("MADReSS_Challenge_2022/data/training-groundtruth.csv")

In [33]:
# here the features are the means and the stds of the file
flattened_feats = []
for fname in training_df["adressfname"]:
    means = pd.read_csv("pyAudioFeatures/%s.csv"%(fname)).mean().add_suffix("mean")
    stds = pd.read_csv("pyAudioFeatures/%s.csv"%(fname)).std().add_suffix("std")
    feats = pd.concat([means,stds])
    feats = means
    feats["fname"] = fname
    flattened_feats.append(feats)
flattened_feats = pd.DataFrame(flattened_feats)

In [34]:
final_feats = training_df[["adressfname","dx","age","gender"]].merge(flattened_feats,left_on="adressfname",right_on="fname",)
final_feats = final_feats.drop(columns=["adressfname","fname"])
final_feats.head()

,dx,age,gender,zcrmean,energymean,energy_entropymean,spectral_centroidmean,spectral_spreadmean,spectral_entropymean,spectral_fluxmean,...,delta chroma_4mean,delta chroma_5mean,delta chroma_6mean,delta chroma_7mean,delta chroma_8mean,delta chroma_9mean,delta chroma_10mean,delta chroma_11mean,delta chroma_12mean,delta chroma_stdmean
0,Control,70,female,0.048794,0.005485,2.533611,0.101507,0.147451,0.184111,0.001663,...,-1.682114e-06,-0.000002,-0.000029,-1.461423e-07,0.000034,5.612216e-05,0.000021,0.000006,1.354871e-07,2.821675e-05
1,Control,72,female,0.044432,0.011529,2.472201,0.088199,0.141110,0.062356,0.001363,...,9.842085e-07,0.000002,0.000039,-7.654574e-06,0.000012,-6.322194e-05,-0.000004,-0.000059,-3.863087e-05,-2.353447e-05
2,Control,74,female,0.046957,0.012607,2.625942,0.120539,0.156420,0.218077,0.001017,...,-2.616079e-05,-0.000007,0.000107,-4.035575e-05,-0.000069,-2.238378e-05,-0.000070,-0.000028,-1.768720e-05,1.683796e-05
3,Control,67,female,0.046704,0.002287,2.573206,0.119546,0.161507,0.210858,0.007507,...,-1.811484e-06,-0.000004,-0.000002,-1.075661e-05,0.000002,-7.427384e-07,-0.000020,-0.000005,-6.571084e-06,-2.931928e-07
4,Control,65,female,0.049657,0.010619,2.601141,0.119435,0.159233,0.193907,0.001151,...,-2.168268e-06,-0.000009,-0.000003,-1.100062e-06,-0.000006,-6.797874e-06,-0.000003,0.000026,5.174994e-06,1.023212e-04


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.inspection import DecisionBoundaryDisplay

clf =  TimeSeriesForestClassifier(n_estimators=5)
   
for fold in range(5):
    final_feats = training_df[["adressfname","dx","age","gender"]].merge(flattened_feats,left_on="adressfname",right_on="fname",)
    final_feats = final_feats.drop(columns=["adressfname"])# 1 fold
#     fold = 4
    test = pd.read_csv("MADReSS_Challenge_2022/data/split/class/fold_%d/test.csv"%fold)
    train = pd.read_csv("MADReSS_Challenge_2022/data/split/class/fold_%d/train.csv"%fold)

    final_feats["gender"] = final_feats["gender"]=="male"
    final_feats["dx"] = final_feats["dx"] == "Control"
    train_df = final_feats[final_feats["fname"].isin(train["adressfname"])]
    test_df = final_feats[final_feats["fname"].isin(test["adressfname"])]
    X_train = train_df.drop(columns=["fname","dx"])
    y_train = train_df["dx"]
    X_test = test_df.drop(columns=["fname","dx"])
    y_test = test_df["dx"]

   
    clf.fit(X_train, y_train)

    y_prob = clf.predict_proba(X_test)
    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))